1


In [199]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import random
import xml.etree.ElementTree as ET 
import csv

MIN_TIME = 1
MAX_TIME = 3


def sleep(MIN_TIME, MAX_TIME):
    time.sleep(random.randint(MIN_TIME*10, MAX_TIME*10)/10)

def build_url(username):
    base_url = "https://www.boardgamegeek.com"
    collections_url = "/xmlapi2/collection?username="
    url = f"{base_url}{collections_url}{username}&excludesubtype=boardgameexpansion&stats=1"

    return url

def userslist():
    usernames_file = "../data/cleaned_usernames.csv"
    usernames = pd.read_csv(usernames_file, names=["username"])\
            .username\
            .values

    try:
        completed_usernames = pd.read_csv("../data/usercomments.csv", names=["username"])\
            .username\
            .unique()
    except pd.errors.EmptyDataError:
        completed_usernames = []
    
    try:
        skipped_usernames = pd.read_csv("../data/usersskipped.csv", names=["username"])\
            .username\
            .unique()
    except pd.errors.EmptyDataError:
        skipped_usernames = []
        
    return list(set(usernames).union(set(skipped_usernames)).difference(set(completed_usernames)))
        

def information(item):
    gameid = item.get("objectid")
    userrating = item.find("stats").find("rating").get("value")
    
    status = item.find("status")
    
    own = status.get("own")
    prevowned = status.get("prevowned")
    fortrade = status.get("fortrade")
    want = status.get("want")
    wanttoplay = status.get("wanttoplay")
    wanttobuy = status.get("wanttobuy")
    wishlist = status.get("wishlist")
    wishlistpriority = status.get("wishlistpriority")
    preordered = status.get("preordered")
    lastmodified = status.get("lastmodified")

    numplays = item.find("numplays").text
    
    comment_tag = item.find("comment")

    if not comment_tag is None:
        commentdata = (username, gameid, userrating, comment_tag.text.encode("utf-8", "ignore"))
    else:
        commentdata = None
    
    userdata = (username, gameid, userrating, own, prevowned, 
            fortrade, want, wanttoplay, wanttobuy, 
            wishlist, wishlistpriority, preordered, 
            numplays, lastmodified)

    return userdata, commentdata

In [198]:
if __name__ == "__main__":
    usernames = userslist()
    
    for i, username in enumerate(usernames[:3]):
        url = build_url(username)
        response = requests.get(url)
        tries = 0
        print(username, response.status_code)
        while not response.status_code == 200:
            response = requests.get(url)
            tries += 1
            sleep(MIN_TIME, MAX_TIME)
            print(username, tries)
            if tries > 2:
                with open("../data/usersskipped.csv", "a+") as f:
                    f.write(f"{username}\n")
                break
                
        data = ET.fromstring(response.text)
        
        timenow = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if "errors" in data.keys():
            message = data.find("message")
            print(message)
        else:
            if data.get("totalitems") != "0":
                items = data.findall("item")
        
                for item in items:
                    userdata, commentdata = information(item)
                    with open("../data/userinfo.csv", "a+") as f:
                        userinfo_out = csv.writer(f)
                        userinfo_out.writerow(userdata)
                        
                    if commentdata:
                        with open("../data/usercomments.csv", "a+") as f:
                            usercomments_out = csv.writer(f)
                            usercomments_out.writerow(commentdata)

            else:
                with open("../data/usernogames.csv", "a+") as f:
                        record = f"{username}\n"
                        f.write(record)
            
                
        sleep(MIN_TIME, MAX_TIME)

griffon8 200
WickedFunGames 200
fourwaychessman 200


In [166]:
item.get("objectid")

'2173'

In [129]:
completed_usernames = [ 'ktusing',
 'Son_of_Sanguinius',
 'wyshmaster6',
 'leannetab90',
 'homerun2777',
 'IndianaGeeking',]

In [110]:
set(usernames).minus(completed_usernames)

AttributeError: 'set' object has no attribute 'minus'

In [111]:
usernameset = set(usernames)

In [113]:
len(usernameset.difference(set(completed_usernames)))

36922

In [151]:
url = build_url("aArendsvark")

In [154]:
data

<Element 'items' at 0x119357590>

In [155]:
data = ET.fromstring(requests.get(url).text)

[<Element 'item' at 0x1193e1590>,
 <Element 'item' at 0x11ad9e110>,
 <Element 'item' at 0x11ad9e770>,
 <Element 'item' at 0x11ad9edd0>,
 <Element 'item' at 0x11ad9c4d0>,
 <Element 'item' at 0x11ad9cb90>,
 <Element 'item' at 0x119dac230>,
 <Element 'item' at 0x119dac890>,
 <Element 'item' at 0x119dacf50>,
 <Element 'item' at 0x119d915f0>,
 <Element 'item' at 0x119d91bf0>,
 <Element 'item' at 0x11980e290>,
 <Element 'item' at 0x11980e8f0>,
 <Element 'item' at 0x11980ef50>,
 <Element 'item' at 0x119d1ad70>,
 <Element 'item' at 0x119812770>,
 <Element 'item' at 0x119812e90>,
 <Element 'item' at 0x105ed6530>]

In [161]:
for item in data.findall("item"):
    print(item.get("objectid"))

94246
31260
1293
13
478
112
131357
2398
133437
153723
149361
168584
84465
134520
102652
103885
123955
9209
